In [ ]:
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import rasterio
from rasterio.plot import show, show_hist

In [ ]:
base_dir = '/home/daric/dev/data/nigeria/geotiffs/'
mul_dir = 'multi_spectral'
#img_file = '19MAY27102816-{}2AS-014909258010_01_P001.TIF' # Invalid
#img_file = '19MAY27102817-{}2AS-014909262010_01_P001.TIF' # Invalid
#img_file = '20FEB22101135-{}2AS-014909251010_01_P001.TIF' # Valid
#img_file = '21FEB25101358-{}2AS-014909264010_01_P001.TIF' # Invalid
#img_file = '21NOV03101658-{}2AS-014909259010_01_P001.TIF' # Invalid
img_file = '21DEC30101738-{}2AS-014909260010_01_P001.TIF' # Valid
#img_file = '22JAN22102131-{}2AS-014909261010_01_P001.TIF' # Valid
mul_file = img_file.format('M')
img_path = os.path.join(base_dir, mul_dir, mul_file)
img = rasterio.open(img_path)
figsize = (12, 12)
fontsize = 25

In [ ]:
blue_band = img.read(2)
green_band = img.read(3)
red_band = img.read(4)
nir_band = img.read(8)

In [ ]:
rgb_img = np.array([red_band, green_band, blue_band]).transpose(1,2,0)

# Plot the RGB image
plt.figure(figsize=figsize)
plt.title('RGB Image\n' + mul_file, fontsize=fontsize)
show(rgb_img.transpose(2,0,1), transform=img.transform)

In [ ]:
# Normalized Difference Index
def compute_ndi(band_a, band_b):
    numerator = band_a - band_b
    denominator = band_a + band_b
    numerator[np.isclose(denominator, np.zeros(denominator.shape))] = 0
    denominator[np.isclose(denominator, np.zeros(denominator.shape))] = 1
    return numerator / denominator

In [ ]:
# Normalized Difference Water Index (NDWI)
ndwi = compute_ndi(green_band, nir_band)
water_thresh = 0.3
water_mask = np.ma.masked_greater_equal(ndwi, water_thresh)

In [ ]:
plt.figure(figsize=figsize)
plt.title('Find Where the River is in the Image\nNDWI (Water Index) above ' + str(water_thresh) + ' threshold\n' + mul_file, fontsize=fontsize)
plt.imshow(water_mask, cmap='jet')

In [ ]:
# Normalized Difference turbidity Index (NDTI)
ndti = compute_ndi(red_band, green_band)
turbid_mask = np.ma.masked_array(ndti, mask=water_mask.mask)

In [ ]:
plt.figure(figsize=figsize)
plt.title('Turbidity for Entire River\nNDTI (Turbidity Index) for River\n' + mul_file, fontsize=fontsize)
plt.imshow(turbid_mask, cmap='jet')

In [ ]:
quant = 0.9
high_turbid_mask = np.ma.masked_less(turbid_mask, np.quantile(turbid_mask, quant))

In [ ]:
plt.figure(figsize=figsize)
plt.title('Show the Most Turbid Areas of the River\nNDTI (Turbidity Index) above ' + str(int(quant * 100)) + 'th percentile\n' + mul_file, fontsize=fontsize)
plt.imshow(high_turbid_mask, cmap='jet')

In [ ]:
red_mask = np.ma.masked_array(red_band, mask=high_turbid_mask.mask)
pd.Series(red_mask[red_mask.mask==False]).describe()

In [ ]:
green_mask = np.ma.masked_array(green_band, mask=high_turbid_mask.mask)
pd.Series(green_mask[green_mask.mask==False]).describe()

In [ ]:
blue_mask = np.ma.masked_array(blue_band, mask=high_turbid_mask.mask)
pd.Series(blue_mask[blue_mask.mask==False]).describe()